Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [133]:
NAME = "Alex Kaariainen"
COLLABORATORS = "CRAIG BEHNKE, ADRIANNA KRIEGER, COLLIN LUFT"

---

# Specifications

Create a data compression encoder and decoder.  The encoder shall output a number of bits within 10% of the source's first-order byte entropy.

An `outbytes, info = encode(seq)` function shall accept a sequence of `bytes` data type of arbitrary length and return a tuple consisting of an encoded sequence of `bytes` and an object describing the encoding method.

A `decode(seq, info)` function shall accept a sequence of `bytes` data type and an object describing the encoding method and return a sequence of `bytes`.

The codec shall have the property that the decoding of an encoded sequence is exactly the same content of the original source sequence.  `decode(encode(source)) == source`

The length of the encoded bytes should be no greater than 1.1 times the length predicted by the source's entropy. 

`entropy(source) * len(source)  <  8 * len(outbytes)  <=  1.1 * entropy(source) * len(source)`

Any self-written compression technique is allowed _except_ for Huffman coding.
It is suggested to use an arithmetic coding method.

## 80% score requirements

Proper function call signatures and data types.  The only requirement for `info` is to not contain a copy of the input source.

* +30: `encode()` returns a tuple of `bytes` and an `object` (anything)
    
Encode --> decode to the same source sequence for input lengths of 27 symbols and fewer.

* +5: 5
* +5: 10
* +5: 20
* +15: 27

Encoded sequence length is within 10% of the first-order entropy for each sequence.

* +5: (8 * len(encoded)) <= (H(source) * len(source)) * 1.5
* +5: (8 * len(encoded)) <= (H(source) * len(source)) * 1.2
* +10: (8 * len(encoded)) <= (H(source) * len(source)) * 1.1

## +10% stretch goal

Encode --> decode works for longer input lengths.

* +2: 100
* +2: 1000
* +2: 10_000
* +2: 100_000
* +2: 1_000_000


## +10% stretch goal

Encode closer to the entropy than 110% for source lengths of 1000 or longer.

* +5: len(encoded) (8 * len(encoded)) <= (H(source) * len(source)) * 1.05
* +5: Average of 3 trials with shorter encoding than a Huffman code

In [134]:
def zero_out(array,length): # zero out the array from 0 to 256
    for i in range(length):  # zero out count array
        array.insert(i,0)
    return array

def cdf(count,cum_count): # populate cum_count array
    for i in range(1,257):
        cum_count[i] += count[i-1]
        cum_count[i] += cum_count[i-1]
    return cum_count

def fill(count,seq):
    total_count = 0
    for byte in seq: # fill count and total_count
        count[byte] += 1
        total_count += 1
    return count,total_count
    
def flipBit(string):
    bits = int(string,base=2)
    bits = bin(int(string,base=2))[2:].zfill(8)
    bitz = bits[0]
    if bitz == '0':
        bits = '1'+string[1:]
    if bitz == '1':
        bits = '0'+string[1:]
    return bits
def giveCoB(string):
    bits = int(string,base=2)
    bits = bin(int(string,base=2))[2:].zfill(8)
    bitz = bits[0]
    if bitz == '0':
        bits = '1'
    if bitz == '1':
        bits = '0'
    return bits

def count_decode(cum_count):
    count=info
    #print("info:",info)
    for i in range(256):
        count[i] = 0
    
    return count

def binary(num):
    byte = bin(num)[2:].zfill(8)
    return byte

In [135]:
import time
from bitarray import bitarray
from math import floor

def encode(seq):
    start_time = time.time()
    print()
    print("-----------------------")
    print("STARTING COMPRESSION...")
    
    count = [] # count:symbol frequency array.
    cum_count = [] # cum_count:cdf array.
    outbytes = [] # outbytes:output sequnce of bytes
    scale3 = 0 # cale3: # of E3 remappings.
    lower = 0 # lower:lower bound
    upper = 255 # upper bound
    
    count = zero_out(count,256) # zero out count array
    count,total_count = fill(count,seq)
    cum_count = zero_out(cum_count, 257) # zero out cum_count array
    cum_count = cdf(count, cum_count) # populate cum_count
    
    #print(" seq is : ", seq)
    print("\t INPUT FILE SIZE: ", total_count, " BYTES")
    
    for byte in seq: # **(get symbol)**loop over input byte seq
        #print("encoding : ", byte)
        bin_low = bin(lower)[2:].zfill(8)
        bin_up = bin(upper)[2:].zfill(8)

        # calculate new upper and lower values to be used in calculations.
        lower_old = lower
        upper_old = upper
        lower = floor(lower_old + ((upper_old - lower_old  +1) * cum_count[byte]) / total_count) # (l)
        upper = floor(lower_old + (((upper_old - lower_old + 1) * cum_count[byte + 1]) / total_count) - 1) # (u)
        
        # convert upper and lower bounds to binary 
        bin_low = bin(lower)[2:].zfill(8)
        bin_up = bin(upper)[2:].zfill(8)
        
        while ( (bin_low[0] == bin_up[0]) or ((bin_low[1] == '1') and (bin_up[1] == '0')) ):
            # convert upper and lower bounds to binary 
            bin_low = bin(lower)[2:].zfill(8)
            bin_up = bin(upper)[2:].zfill(8)
            
            if bin_low[0] == bin_up[0]: 
               # print("equal")
                # shift msb out to the output
                outbytes.append(bin_low[0]) # **(send b)** 
                #print("SHIFT OUT = ", bin_low[0])
                lower = int(bin_low[1:8] + '0', base = 2) # **(shift l to the left by 1 bit and shift 0 into LSB)**
                upper = int(bin_up[1:8] + '1', base = 2) # **(shift u to the left by 1 bit and shft 1 inot LSB)**
                
                while scale3 > 0:
                   # print("scale3: ",giveCoB(bin_low))
                    outbytes.append(giveCoB(bin_low))# **(send complement of b)**
                    scale3 += -1 # **(decrement scale3)**
                    
            else:# **(E3 condition holds)** may need to switch lower and upper 
                if ((bin_low[1] == '1') and (bin_up[1] == '0') ): # check second bits
                    #print("**SHIFT E3**")
                    lower = int(bin_low[0] + bin_low[2:8] + '0', base = 2) # **(shift l to the left by 1 bit and shift 0 into LSB)**
                    upper = int(bin_up[0] + bin_up[2:8] + '1', base = 2) # **(shift u to the left by 1 bit and shft 1 inot LSB)**
                    bin_low = bin(lower)[2:].zfill(8)
                    bin_up = bin(upper)[2:].zfill(8)
                    scale3 += 1 # **(increment scale3)**
                   
    outbytes.append(bin_low[0])
   # if(scale3==1):
    #    outbytes.append(str(1-int(bin_low[0])))
    while scale3 > 0:
        outbytes.append('1')
        scale3 += -1
        
    outbytes.append(bin_low[1:8])
    
    out = "".join(outbytes)
    
    output = bitarray(out)
    outbytes = out
    outstuff = output.tobytes()
    #print("outstuff:",outstuff)
    
    #for i in range(len(outbytes)):
   #     out += outbytes[i]

   # out = bytes(out,'utf8')
    outsize = (len(out)/8)
    end_time = time.time()
    print("\t OUTPUT FILE SIZE : ", round(outsize), " BYTES.")
    print("\t COMPRESSION RATIO : {:.1f}".format(total_count/outsize), " INPUT BYTES PER OUTPUT BYTES." )
    print("\t TIME ELAPSED {:.5f}".format((end_time-start_time)), "SECONDS.")
    print("DONE COMPRESSING...")
   # print("-----------------------")
    print()
    return (outstuff, cum_count)

In [136]:
import time

def decode(seq, info):
    start_time = time.time()
    #print(type(seq))
    print()
    #print("-----------------------")
    print("BEGINNING DECODING CALCULATIONS... ")
    
    cum_count = info # cum_count:cdf array.
    message = "" # outbytes:output sequnce of bytes
    lower = 0 # lower:lower bound
    upper = 255 # upper bound
   # print(" lower init : ", lower, " upper init : ", upper)
    total_count = info[len(info)-1]
    #print("total count:",total_count)
    m = 8 # number of bits in buffer
    offset=0
    #index = 1
    
    #print("BEGINNING DECODING SEQUENCE: ", seq)
    output = bytearray()
    
    k = 0
    #print(type(seq))
    
   # print(bytes(seq))
    
    #print("BEGINNING DECODING SEQUENCE: ", seq)
    for byte in seq:
       # print("message:",message)
       # print("bin (byte),",binary(byte))
       # print("seq:",seq)
       # print("byte:",byte)
        message = message + binary(byte)
       # print("------------")
  #  print("message post:",message)
    
    for i in range(total_count):
        
        t_bin = message[offset:offset+8] # **(read first m bits of received bitstream into tag t)**
       # print("tag:",t_bin, " ", int(t_bin,base=2))
        t = int(t_bin,base=2)
        
       # print("t:",t)
        #print("lower:",lower)
       # print("total_count:",total_count)
       # print("upper:",upper)
        x = ( (((t - lower + 1) * total_count) - 1 ) / (upper - lower + 1))
       # print("x:",x)
        
        for j in range(256):
           # print("j=",j)
           # print("is ",x, " > ",cum_count[j])
            if x >= cum_count[j]:
               # print("yes")
                varx = 0
            else:
               # print("no")
                output.append(j-1)
                
               # print("decoded: ",output)
                
                lower_old = lower
                upper_old = upper
                
                lower = floor(lower_old + (( upper_old - lower_old + 1 ) * cum_count[j-1]) / total_count)
                upper = floor(lower_old + ((( upper_old - lower_old + 1 ) * cum_count[j]) / total_count) - 1)
                
                bin_low = binary(lower)
                bin_up = binary(upper)
                
               # print("upper:",upper, " ",bin_up)
              #  print("lower:",lower, " ",bin_low)
                
                # COMPARE MSB OF LOWER AND UPPER
                while(True):
                    if bin_low[0] == bin_up[0]:
                        
                        #shift out msb
                        lower = int(bin_low[1:8] + '0',base = 2)
                        upper = int(bin_up[1:8] + '1', base = 2)
                        bin_low = binary(lower)
                        bin_up = binary(upper)
                        
                      #  print("shift out")
                      #  print("lower: ",lower, " ",bin_low)
                      #  print("upper: ",upper, " ",bin_up)
                        offset += 1
                    else:
                        break
                        
                #check E3
                while(True):
                    bin_low = binary(lower)
                    bin_up = binary(upper)
                    
                    if ( (bin_low[1] == '1') and (bin_up[1] == '0')):
                       # print("**SHIFT E3**")
                        lower = int(bin_low[0] + bin_low[2:8] + '0', base = 2)
                        upper = int(bin_up[0] + bin_up[2:8] + '1', base = 2)
                        bin_low = binary(lower)
                        bin_up = binary(upper)
                       # print("lower: ",lower, " ", bin_low)
                       # print("upper: ",upper, " ", bin_up)
                        
                        message = message[:offset] + str(1 - int(message[offset+1])) + message[offset+2:]
                        tag = message[offset:offset+m]
                       # print("tag:",tag)
                        int_tag = int(tag,base = 2)
                       # print("int tag: ",int_tag, type(int_tag))
                        
                    else:
                        #print("final lower: ", lower, " ", bin_low)
                       # print("final upper: ", upper, " ", bin_up)
                        break
                break
            
    #print("before: ",output)
    output = bytes(output)
    #print("final answer is: ",output)
    #print("")
   # print()
    end_time = time.time()
    print("\t INPUT FILE SIZE : ",len(seq), " BYTES.")
    print("\t OUTPUT FILE SIZE : ", len(output), " BYTES.")
    print("\t TIME ELAPSED {:.5f}".format((end_time-start_time)), "SECONDS.")
    print("DONE DECODING.")
    print("-----------------------")
    print()
    return output
                

In [137]:
#define encode(seq):
#    
#    return out,info

In [138]:
seq = b'\x01\x03\x02\x01\xa1'
#seq = b'\xFF\x30\x99\x00\x01'
#print("seq : ",seq)
out,info = encode(seq)
#print("FINAL SEQUENCE : ", out)
end_seq = decode(out,info)


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  5  BYTES
	 OUTPUT FILE SIZE :  2  BYTES.
	 COMPRESSION RATIO : 2.4  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00057 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  3  BYTES.
	 OUTPUT FILE SIZE :  5  BYTES.
	 TIME ELAPSED 0.00015 SECONDS.
DONE DECODING.
-----------------------



In [139]:
# read-only cell
import source
import huffman

In [140]:
# 30 points
# check the data types of the encoder
s = source.generate(10)
e = encode(s)   # capture the entire return value
encbytes, info = e   # split into the two parts

assert(type(e) == tuple)
assert(type(encbytes) == bytes)
assert(isinstance(info, object))

# check decoder output format
out = decode(encbytes, info)
assert(type(out) == bytes)


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  10  BYTES
	 OUTPUT FILE SIZE :  4  BYTES.
	 COMPRESSION RATIO : 2.3  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00062 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  5  BYTES.
	 OUTPUT FILE SIZE :  10  BYTES.
	 TIME ELAPSED 0.00016 SECONDS.
DONE DECODING.
-----------------------



In [141]:
# read-only utility function
from copy import copy

def verify_roundtrip(N, trials=10):
    print('Checking input length: {}'.format(N))
    # check for several random source sequences
    for _ in range(trials):
        s = source.generate(N)
        e, info = encode(s)
        original = copy(s)
        del s
        out = decode(e, info)
        assert(out == original)
    print('ok')
        
    

In [142]:
# 5 points
verify_roundtrip(5)

Checking input length: 5

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  5  BYTES
	 OUTPUT FILE SIZE :  2  BYTES.
	 COMPRESSION RATIO : 2.4  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00054 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  3  BYTES.
	 OUTPUT FILE SIZE :  5  BYTES.
	 TIME ELAPSED 0.00013 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  5  BYTES
	 OUTPUT FILE SIZE :  2  BYTES.
	 COMPRESSION RATIO : 2.5  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00054 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  2  BYTES.
	 OUTPUT FILE SIZE :  5  BYTES.
	 TIME ELAPSED 0.00014 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  5  BYTES
	 OUTPUT FILE SIZE :  2  BYTES.
	 COMPRESSION RATIO : 2.5  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00738 S

In [143]:
# 5 points
verify_roundtrip(10)

Checking input length: 10

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  10  BYTES
	 OUTPUT FILE SIZE :  5  BYTES.
	 COMPRESSION RATIO : 2.2  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00085 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  5  BYTES.
	 OUTPUT FILE SIZE :  10  BYTES.
	 TIME ELAPSED 0.00020 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  10  BYTES
	 OUTPUT FILE SIZE :  4  BYTES.
	 COMPRESSION RATIO : 2.5  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00044 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  4  BYTES.
	 OUTPUT FILE SIZE :  10  BYTES.
	 TIME ELAPSED 0.00016 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  10  BYTES
	 OUTPUT FILE SIZE :  4  BYTES.
	 COMPRESSION RATIO : 2.6  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.0

In [144]:
# 5 points
verify_roundtrip(20)

Checking input length: 20

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  20  BYTES
	 OUTPUT FILE SIZE :  10  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00067 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  11  BYTES.
	 OUTPUT FILE SIZE :  20  BYTES.
	 TIME ELAPSED 0.00069 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  20  BYTES
	 OUTPUT FILE SIZE :  11  BYTES.
	 COMPRESSION RATIO : 1.9  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00085 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  11  BYTES.
	 OUTPUT FILE SIZE :  20  BYTES.
	 TIME ELAPSED 0.00035 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  20  BYTES
	 OUTPUT FILE SIZE :  10  BYTES.
	 COMPRESSION RATIO : 2.1  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSE

In [145]:
# 15 points
verify_roundtrip(27)

Checking input length: 27

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 1.9  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00100 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  14  BYTES.
	 OUTPUT FILE SIZE :  27  BYTES.
	 TIME ELAPSED 0.00060 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00060 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  14  BYTES.
	 OUTPUT FILE SIZE :  27  BYTES.
	 TIME ELAPSED 0.00047 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSE

In [146]:
# read-only
# reference codec

def huffencode(seq):
    from collections import Counter
    import huffman
    
    counts = Counter(seq)
    
    # add an END-OF-SEQUENCE symbol
    counts[257] = 1

    tree = huffman.huffTree(counts)
    huffcode = huffman.huffCode(tree)
    
    out = bitarray()
    out.encode(huffcode, seq)
    
    # encode the END-OF-SEQUENCE symbol
    out.encode(huffcode, [257])
    
    outbytes = out.tobytes()
    info = huffcode
    return (outbytes, info)

def huffdecode(seq, info):
    import huffman
    bits = bitarray()
    bits.frombytes(seq)
    
    tree = huffman.make_tree(info)
    out = huffman.decode(tree, bits)
    
    # find the END-OF-SEQUENCE symbol
    idx = out.index(257)
    
    # trim off the END-OF-SEQUENCE symbol and following bytes
    return bytes(out[:idx])

In [147]:
# read-only utility functions

from collections import Counter
from math import ceil

def entropy(s):
    counts = Counter(s)
    return source.entropy(list(counts.values()))

def verify_compression(max_ratio, N=27, trials=5):
    encoded = 0
    expected = 0
    huff = 0
    print('Source length = {}'.format(N))
    print(' #    H       enc     min   ratio')
    for trial in range(trials):
        s = source.generate(N)
        Hsource = entropy(s)
        e, info = encode(s)

        encoded_bits = 8 * len(e)
        expected_bits = ceil(Hsource * N)
        excess_ratio = (encoded_bits / expected_bits)
        huff_bits = 8 * len(huffencode(s)[0])

        encoded += encoded_bits
        expected += expected_bits
        huff += huff_bits

        if trials <= 10 or (trial % 10) == 0:
            print('{:2d}  {:.4f}  {:6d}  {:6d}  {:.3f}'.format(
                    trial, Hsource, encoded_bits, expected_bits, excess_ratio))
    
    print('---------------------------------')
    codec_ratio = encoded / expected
    huff_ratio = huff / expected
    print('Avg ratio = {:.3f}'.format(codec_ratio))
    assert(codec_ratio <= max_ratio)
    return codec_ratio, huff_ratio
    
    

In [148]:
# 5 points
verify_compression(1.5)

Source length = 27
 #    H       enc     min   ratio

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  15  BYTES.
	 COMPRESSION RATIO : 1.8  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00100 SECONDS.
DONE COMPRESSING...

 0  4.2545     128     115  1.113

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00063 SECONDS.
DONE COMPRESSING...

 1  3.6619     112      99  1.131

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.1  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00113 SECONDS.
DONE COMPRESSING...

 2  3.5003     104      95  1.095

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 1.9  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELA

(1.111545988258317, 1.111545988258317)

In [149]:
# 5 points
verify_compression(1.2)

Source length = 27
 #    H       enc     min   ratio

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00154 SECONDS.
DONE COMPRESSING...

 0  3.6899     112     100  1.120

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 1.9  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00065 SECONDS.
DONE COMPRESSING...

 1  3.9862     120     108  1.111

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.1  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00071 SECONDS.
DONE COMPRESSING...

 2  3.6023     112      98  1.143

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  15  BYTES.
	 COMPRESSION RATIO : 1.8  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELA

(1.114119922630561, 1.114119922630561)

In [150]:
# 10 points

# this is difficult to reliably test using only 27 input symbols
# --> too much variance
verify_compression(1.101, trials=100)

Source length = 27
 #    H       enc     min   ratio

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 1.9  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00089 SECONDS.
DONE COMPRESSING...

 0  3.8380     112     104  1.077

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.1  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00399 SECONDS.
DONE COMPRESSING...


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00228 SECONDS.
DONE COMPRESSING...


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00095 SECONDS.
DONE COMPRESSING...


-----------------------


	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00103 SECONDS.
DONE COMPRESSING...


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 1.9  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00069 SECONDS.
DONE COMPRESSING...


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  13  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00052 SECONDS.
DONE COMPRESSING...


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 2.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00052 SECONDS.
DONE COMPRESSING...


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  27  BYTES
	 OUTPUT FILE SIZE :  14  BYTES.
	 COMPRESSION RATIO : 1.9  INPUT BYTES PER OUTPUT

(1.0983129726585223, 1.0983129726585223)

## Test longer input lengths

In [151]:
# 2 points
verify_roundtrip(100)

Checking input length: 100

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  100  BYTES
	 OUTPUT FILE SIZE :  57  BYTES.
	 COMPRESSION RATIO : 1.7  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00314 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  58  BYTES.
	 OUTPUT FILE SIZE :  100  BYTES.
	 TIME ELAPSED 0.00209 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  100  BYTES
	 OUTPUT FILE SIZE :  57  BYTES.
	 COMPRESSION RATIO : 1.7  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00218 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 
	 INPUT FILE SIZE :  58  BYTES.
	 OUTPUT FILE SIZE :  100  BYTES.
	 TIME ELAPSED 0.00189 SECONDS.
DONE DECODING.
-----------------------


-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  100  BYTES
	 OUTPUT FILE SIZE :  58  BYTES.
	 COMPRESSION RATIO : 1.7  INPUT BYTES PER OUTPUT BYTES.
	 TIME 

ValueError: invalid literal for int() with base 2: ''

In [152]:
# 2 points
verify_roundtrip(1000)

Checking input length: 1000

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  1000  BYTES
	 OUTPUT FILE SIZE :  9  BYTES.
	 COMPRESSION RATIO : 112.7  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.00449 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 


ValueError: invalid literal for int() with base 2: ''

In [153]:
# 2 points
verify_roundtrip(10_000)

Checking input length: 10000

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  10000  BYTES
	 OUTPUT FILE SIZE :  3  BYTES.
	 COMPRESSION RATIO : 2963.0  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 0.03014 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 


ValueError: invalid literal for int() with base 2: ''

In [ ]:
# 2 points
verify_roundtrip(100_000, trials=2)

In [154]:
# 2 points
verify_roundtrip(1_000_000, trials=1)

Checking input length: 1000000

-----------------------
STARTING COMPRESSION...
	 INPUT FILE SIZE:  1000000  BYTES
	 OUTPUT FILE SIZE :  2  BYTES.
	 COMPRESSION RATIO : 444444.4  INPUT BYTES PER OUTPUT BYTES.
	 TIME ELAPSED 2.38009 SECONDS.
DONE COMPRESSING...


BEGINNING DECODING CALCULATIONS... 


ValueError: invalid literal for int() with base 2: ''

## Test encoding closer to entropy

For 1000 source symbols.

In [ ]:
# 5 points
verify_compression(1.05, N=1000, trials=10)

In [ ]:
# 5 points
codec_ratio, huff_ratio = verify_compression(1.1, N=1000, trials=10)

print()
print('Codec:   {:.5f}'.format(codec_ratio))
print('Huffman: {:.5f}'.format(huff_ratio))

assert(codec_ratio <= huff_ratio)